In [3]:
import sys
import os
import pandas as pd
import streamlit as st
import yaml

# Add project root to sys.path in Jupyter or interactive session
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../")))

from config.path_config import DATA_DIR, PROJECT_DIR, DEPENDENCIES_DIR
from src.helper_functions.data_pre_processing.data_upload_processor.processor import *
from src.helper_functions.database_upload.postgres_uploader import *

from dotenv import load_dotenv

In [4]:
yaml_file_path = "testing_mapping.yaml"
print(yaml_file_path)

testing_mapping.yaml


In [8]:
raw_data = pd.read_excel(os.path.join(DATA_DIR, "wave_10_raw_data.xlsx"))
data_map = pd.read_excel(os.path.join(DATA_DIR, "wave_10_data_map.xlsx"))

In [5]:
engine = create_postgres_engine('postgres', 'postgres', 'localhost', 5432, 'cl_survey_data')

In [7]:
env_path = os.path.join(DEPENDENCIES_DIR, ".env")
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")

In [9]:
data_dict = run_question_data_pipeline(data_map, raw_data, api_key,'wave_10')

In [10]:
data_dict.keys()

dict_keys(['question_guide', 'mapped_data', 'type_subtype', 'question_dict', 'embeddings_metadata_df', 'embedding_df'])

In [19]:
data_dict['embedding_df']

,primary_key,chunk_embedding
0,wave_10_0,"[0.009160860125099458, -0.012211981738858768, ..."
1,wave_10_1,"[-0.001735233869475999, -0.003143990760027482,..."
2,wave_10_2,"[0.0016934563369195971, -0.003148743004308896,..."
3,wave_10_3,"[0.005397405175799898, 0.010119272765629845, -..."
4,wave_10_4,"[0.01174797519237197, -0.010960044609830894, -..."
...,...,...
73,wave_10_73,"[-0.029641089178786578, -0.007390699618755048,..."
74,wave_10_74,"[-0.026912961709606834, -0.023842547580038904,..."
75,wave_10_75,"[-0.011305906926328437, -0.041125692850793615,..."
76,wave_10_76,"[0.0037498873825671402, -0.04143672981848321, ..."


In [12]:
push_dataframe_to_postgres_db(data_dict['embeddings_metadata_df'], 'wave_9_embeddings_metadata', engine)
push_dataframe_to_postgres_db(data_dict['type_subtype'], file_name='wave_9_type_subtype', engine=engine)
push_dataframe_to_postgres_db(data_dict['question_guide'], 'wave_9_question_guide', engine)
process_raw_data(data_dict['mapped_data'], 'wave_9', engine, response_id_col='Respondent', chunk_size=60)

2025-04-11 13:45:40.171 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:45:40.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

{'Q8_000': 'Q8.Improved_quality_of_life__e.g.__closer_to_nature__family__shorter_commute_', 'Q14_001': 'Q14.Live_with_extended_family___relatives__e.g.__siblings__parents__grandparents__other_relatives_', 'Q25_002': 'Q25.Large_tech_companies__i.e._Google__Amazon__Apple__Microsoft__Facebook__Twitter_', 'Q25_003': 'Q25.Traditional_media__i.e._CBS__Fox__BBC__France2__ZDF__Telecinco_', 'Q26_004': 'Q26.I_am_currently_postponing_making_large_expenditures__e.g.__home_buying__home_improvement__until_I_know_the_results_of_the_US_election', 'Q26_005': 'Q26._I_am_currently_postponing_making_financial_investments__e.g.__stock_purchases__until_I_know_the_results_of_the_US_election', 'Q26_006': 'Q26.I_am_currently_reducing_my_day_to_day_spending_until_I_know_the_results_of_the_US_election', 'Q26_007': 'Q26.I_am_currently_postponing_moving___living_situation_decisions__e.g.__country__state__until_I_know_the_results_of_the_US_election', 'Q26_008': 'Q26.I_am_currently_postponing_work_decisions__e.g.__c

2025-04-11 13:46:16.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-11 13:46:16.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [21]:
data_dict['embedding_df'].head(5)

,primary_key,chunk_embedding
0,wave_10_0,"[0.009160860125099458, -0.012211981738858768, ..."
1,wave_10_1,"[-0.001735233869475999, -0.003143990760027482,..."
2,wave_10_2,"[0.0016934563369195971, -0.003148743004308896,..."
3,wave_10_3,"[0.005397405175799898, 0.010119272765629845, -..."
4,wave_10_4,"[0.01174797519237197, -0.010960044609830894, -..."


In [26]:
import psycopg2
from psycopg2.extras import execute_batch

def store_embeddings_to_pgvector(df: pd.DataFrame, db_config: dict, table_name: str, label_column="primary_key", embedding_column="chunk_embedding"):
    """
    Store a pandas DataFrame with vector embeddings into a PostgreSQL table using PGVector.

    Parameters:
        df (pd.DataFrame): DataFrame with a label and an embedding column.
        db_config (dict): Dictionary with keys: dbname, user, password, host, port.
        table_name (str): Name of the table to store data in.
        label_column (str): Column in df that contains the labels (e.g. text).
        embedding_column (str): Column in df that contains the embeddings (list of floats).
    """

    # Get embedding dimension
    sample_embedding = df.iloc[0][embedding_column]
    vector_dim = len(sample_embedding)

    # Connect to database
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()

    # Ensure PGVector extension is enabled
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

    # Create table
    cur.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id SERIAL PRIMARY KEY,
            {label_column} TEXT,
            {embedding_column} VECTOR({vector_dim})
        );
    """)

    table_name = table_name + "_embeddings"
    # Insert data using batch insert
    rows = [(row[label_column], row[embedding_column]) for _, row in df.iterrows()]
    execute_batch(cur, f"""
        INSERT INTO {table_name} ({label_column}, {embedding_column})
        VALUES (%s, %s)
    """, rows)

    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Successfully inserted {len(rows)} records into '{table_name}'.")



In [24]:
db_config = {
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": 5432,
    "dbname": "cl_survey_data"
}


In [25]:
store_embeddings_to_pgvector(data_dict['embedding_df'], db_config, 'wave_9_embeddings', label_column='primary_key', embedding_column='chunk_embedding')

FeatureNotSupported: extension "vector" is not available
DETAIL:  Could not open extension control file "C:/Program Files/PostgreSQL/17/share/extension/vector.control": No such file or directory.
HINT:  The extension must first be installed on the system where PostgreSQL is running.
